#Files needed
* CNN_CIFAR10_net3.h5
* adv_de_cnn_cifar_ssim
* subset_cifar

# Import Libraries

In [1]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import cauchy 
from scipy import stats
import operator
import random
from sklearn.metrics import accuracy_score
from skimage.measure import compare_ssim as ssim
import pickle
import cv2
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation

#Set seeds
random.seed(0)
np.random.seed(0)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Get the data from pickle files

In [0]:

original = None
adversarial = None 
original_y = None
adversarial_y = None
dump = None
adversarial_rbf = None
adversarial_rbf_y  = None
adversarial_cnn = None
adversarial_cnn_y  = None

with open('subset_cifar', 'rb') as f:
    original = pickle.load(f)
    adversarial= pickle.load(f)
    original_y = pickle.load(f)
    adversarial_y = pickle.load(f)

with open('adv_de_cnn_cifar_ssim', 'rb') as f:
    adversarial_cnn = pickle.load(f)
    for i in range(3):
      dump = pickle.load(f)
    adversarial_cnn_y= pickle.load(f)

In [3]:
original.shape

(914, 32, 32, 1)

In [4]:
adversarial.shape

(133, 32, 32, 1)

In [5]:
adversarial_cnn.shape

(133, 32, 32, 1)

# Calculate PSNR and SSIM

In [0]:
psnrs_cnn = []
for i in range(adversarial.shape[0]):
  psnr_ = cv2.PSNR(adversarial[i].reshape(32,32), adversarial[i].reshape(32,32))
  psnrs_cnn.append((cv2.PSNR(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32))/psnr_)*100)

In [7]:
#double check GeeksForGeeks psnr
print(np.around(np.mean(psnrs_cnn),3), "+-" ,np.around(np.std(psnrs_cnn),3),"(" ,np.around(np.min(psnrs_cnn),3), ",",
                                                                                        np.around(np.max(psnrs_cnn),3),")")

7.623 +- 1.605 ( 3.278 , 10.007 )


In [8]:
ssims_cnn = []
for i in range(adversarial.shape[0]):
  ssims_cnn.append(ssim(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
print(np.around(np.mean(ssims_cnn),3), "+-" ,np.around(np.std(ssims_cnn),3),"(" ,np.around(np.min(ssims_cnn),3), ",",
                                                                                        np.around(np.max(ssims_cnn),3),")")

0.855 +- 0.141 ( 0.264 , 0.984 )


# Prepare Data

In [0]:
threshold_psnr = 7
threshold_ssim = 0.7
for i in range(len(adversarial)):
  if psnrs_cnn[i] < threshold_psnr or ssims_cnn[i] < threshold_ssim:
    adversarial_cnn[i] = adversarial[i]

  

In [0]:
#Normalize images
original_sc = original        /255.0
adversarial_sc = adversarial     /255.0 
adversarial_cnn_sc = adversarial_cnn /255.0

# Predict Using CNN

In [0]:
from keras.models import load_model
model = load_model("CNN_CIFAR10_net3.h5")

In [0]:
#Get predictions
cnn_pred_original = model.predict(original_sc)
cnn_pred_adversarial = model.predict(adversarial_sc)
cnn_pred_adversarial_cnn = model.predict(adversarial_cnn_sc)

In [14]:
print("Accuracy on original: ",accuracy_score(original_y,np.argmax(cnn_pred_original,axis=1)))
print("Accuracy on adversarial: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial,axis=1)))
print("Accuracy on adversarial_cnn: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial_cnn,axis=1)))

Accuracy on original:  0.824945295404814
Accuracy on adversarial:  0.8195488721804511
Accuracy on adversarial_cnn:  0.2556390977443609


In [15]:
print("Accuracy on subset: ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (CNN): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial_cnn,axis=0),axis=1)))

Accuracy on subset:  0.8242597898758357
Accuracy on subset with adversarial examples (CNN):  0.7526265520534862
